In [1]:
import pandas as pd
import numpy as np
import datetime

# Q9

In [1]:
import pandas as pd  # Import pandas for data manipulation
import numpy as np  # Import numpy for numerical operations

def calculate_distance_matrix(df) -> pd.DataFrame:
    # Define a function to calculate the distance matrix for toll IDs.

    toll_ids = pd.concat([df['id_start'], df['id_end']]).unique()  
    # Combine the 'id_start' and 'id_end' columns, and get all unique toll IDs from both columns.

    distance_matrix = pd.DataFrame(np.inf, index=toll_ids, columns=toll_ids, dtype=float)  
    # Create an empty distance matrix, initially filled with infinity values, where rows and columns are toll IDs.

    np.fill_diagonal(distance_matrix.values, 0)  
    # Set the diagonal values (distance from a toll ID to itself) to 0, because the distance from a point to itself is 0.

    # Iterate through each row of the DataFrame to fill the distance matrix with actual distances.
    for _, row in df.iterrows():  
        # Loop through each row in the DataFrame.
        
        source = row['id_start']  # Get the start toll ID.
        destination = row['id_end']  # Get the end toll ID.
        distance = row['distance']  # Get the distance between the start and end toll IDs.

        distance_matrix.at[source, destination] = distance  # Set the distance for the source -> destination direction.
        distance_matrix.at[destination, source] = distance  # Set the distance for the destination -> source direction (undirected graph).

    # Now apply the Floyd-Warshall algorithm to find the shortest path between all pairs of toll IDs.
    for k in toll_ids:  # Loop through each toll ID (acting as an intermediate node).
        for i in toll_ids:  # Loop through all toll IDs for the source.
            for j in toll_ids:  # Loop through all toll IDs for the destination.
                # If a shorter path from i to j exists through k, update the distance matrix.
                if distance_matrix.at[i, j] > distance_matrix.at[i, k] + distance_matrix.at[k, j]:
                    distance_matrix.at[i, j] = distance_matrix.at[i, k] + distance_matrix.at[k, j]

    return distance_matrix  # Return the final distance matrix with shortest paths.



In [2]:
df = pd.read_csv('dataset-2.csv')
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


In [3]:
distance_matrix = calculate_distance_matrix(df)
print("Distance Matrix:")
print(distance_matrix.head())

Distance Matrix:
         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   

         1001414  1001416  1001418  ...  1001458  1001460  1001461  1001462  \
1001400    112.5    125.7    139.3  ...    348.8    353.9    366.7    371.8   
1001402    102.8    116.0    129.6  ...    339.1    344.2    357.0    362.1   
1001404     82.6     95.8    109.4  ...    318.9    324.0    336.8    341.9   
1001406     66.6     79.8     93.4  ...    302.9    308.0    320.8    325.9   
1001408     44.9     58.1     71.7  ...    281.2    286.3    299.1    304.2   

         1001464  1001466  1001468  1001470  1001437  1001472  

# Q10

In [4]:
import pandas as pd  # Import pandas for data manipulation

def unroll_distance_matrix(df) -> pd.DataFrame:
    # Define a function to "unroll" the distance matrix into a long format (flattening it).

    unrolled_data = []  # Create an empty list to store the unrolled data.

    # Loop through each row (id_start) in the DataFrame.
    for id_start in df.index:
        # Loop through each column (id_end) in the DataFrame.
        for id_end in df.columns:
            # Skip if the start and end IDs are the same (no need to include distances for the same toll ID).
            if id_start != id_end:
                distance = df.at[id_start, id_end]  # Get the distance value for the pair (id_start, id_end).
                unrolled_data.append([id_start, id_end, distance])  # Append the data as a list to the unrolled_data list.

    # Convert the unrolled_data list into a DataFrame with columns: 'id_start', 'id_end', and 'distance'.
    unrolled_df = pd.DataFrame(unrolled_data, columns=['id_start', 'id_end', 'distance'])

    return unrolled_df  # Return the unrolled DataFrame.


In [5]:
unrolled_df = unroll_distance_matrix(distance_matrix)
print("\nUnrolled Distance Matrix:")
print(unrolled_df)


Unrolled Distance Matrix:
      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1001472  1001464      45.8
1802   1001472  1001466      37.3
1803   1001472  1001468      26.6
1804   1001472  1001470      16.0
1805   1001472  1001437     202.2

[1806 rows x 3 columns]


# Q11

In [6]:
import pandas as pd  # Import pandas for data manipulation

def find_ids_within_ten_percentage_threshold(df, reference_id) -> pd.DataFrame:
    # Define a function to find toll IDs whose average distance is within 10% of a reference toll ID.

    # Calculate the average distance for the reference_id (distance of all rows where id_start equals reference_id).
    reference_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()
    
    # Define the floor and ceiling thresholds (±10% of the reference average distance).
    threshold_floor = reference_avg_distance * 0.9
    threshold_ceiling = reference_avg_distance * 1.1 
    
    matching_ids = []  # Create an empty list to store matching toll IDs.
    
    # Loop through each unique toll ID (id_start) in the DataFrame.
    for id_start in df['id_start'].unique():
        # Calculate the average distance for each toll ID.
        avg_distance = df[df['id_start'] == id_start]['distance'].mean()
        
        # Check if the average distance is within the ±10% range of the reference toll ID's average distance.
        if threshold_floor <= avg_distance <= threshold_ceiling:
            matching_ids.append(id_start)  # Add the matching ID to the list.

    return sorted(matching_ids)  # Return the list of matching toll IDs, sorted in ascending order.


In [7]:
reference_id = 1001416
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print("\nIDs within 10% of average distance for reference ID {}:".format(reference_id))
print(ids_within_threshold)


IDs within 10% of average distance for reference ID 1001416:
[1001414, 1001416, 1001418, 1001458, 1001460, 1001461, 1001462]


# Q12

In [8]:
import pandas as pd  # Import pandas for data manipulation

def calculate_toll_rate(df) -> pd.DataFrame:
    # Define a function to calculate toll rates for different vehicle types based on distance.

    df['moto'] = df['distance'] * 0.8  # Calculate toll rate for motorcycles (distance * 0.8).
    df['car'] = df['distance'] * 1.2    # Calculate toll rate for cars (distance * 1.2).
    df['rv'] = df['distance'] * 1.5     # Calculate toll rate for RVs (distance * 1.5).
    df['bus'] = df['distance'] * 2.2    # Calculate toll rate for buses (distance * 2.2).
    df['truck'] = df['distance'] * 3.6  # Calculate toll rate for trucks (distance * 3.6).

    return df  # Return the DataFrame with the new toll rate columns.


In [9]:
toll_df = calculate_toll_rate(unrolled_df)
print("\nToll Rates:")
print(toll_df)


Toll Rates:
      id_start   id_end  distance    moto     car      rv     bus   truck
0      1001400  1001402       9.7    7.76   11.64   14.55   21.34   34.92
1      1001400  1001404      29.9   23.92   35.88   44.85   65.78  107.64
2      1001400  1001406      45.9   36.72   55.08   68.85  100.98  165.24
3      1001400  1001408      67.6   54.08   81.12  101.40  148.72  243.36
4      1001400  1001410      78.7   62.96   94.44  118.05  173.14  283.32
...        ...      ...       ...     ...     ...     ...     ...     ...
1801   1001472  1001464      45.8   36.64   54.96   68.70  100.76  164.88
1802   1001472  1001466      37.3   29.84   44.76   55.95   82.06  134.28
1803   1001472  1001468      26.6   21.28   31.92   39.90   58.52   95.76
1804   1001472  1001470      16.0   12.80   19.20   24.00   35.20   57.60
1805   1001472  1001437     202.2  161.76  242.64  303.30  444.84  727.92

[1806 rows x 8 columns]


# Q13

In [10]:
import pandas as pd  # Import pandas for data manipulation
import datetime  # Import datetime to handle time objects

def calculate_time_based_toll_rates(df) -> pd.DataFrame:
    # Define a function to calculate toll rates based on time of day and day of the week.

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  # List of days in a week
    time_ranges = [
        # Define time ranges and corresponding toll rate multiplier for weekdays
        (datetime.time(0, 0), datetime.time(10, 0), 0.8),   # From midnight to 10:00 AM, 0.8 multiplier
        (datetime.time(10, 0), datetime.time(18, 0), 1.2),  # From 10:00 AM to 6:00 PM, 1.2 multiplier
        (datetime.time(18, 0), datetime.time(23, 59, 59), 0.8)  # From 6:00 PM to midnight, 0.8 multiplier
    ]

    new_rows = []  # Create an empty list to store the new rows with time-based toll rates

    # Loop through each row in the DataFrame to apply the time-based toll rates
    for _, row in df.iterrows():
        for day in days:  # Loop through each day of the week
            for start_time, end_time, weekday_discount in time_ranges:  # Loop through each time range
                new_row = row.copy()  # Create a copy of the current row to avoid modifying the original

                # Assign the current day to the 'start_day' and 'end_day' columns
                new_row['start_day'] = day
                new_row['end_day'] = day
                
                # Assign the current time range to the 'start_time' and 'end_time' columns
                new_row['start_time'] = start_time
                new_row['end_time'] = end_time

                # Apply a discount based on whether the day is a weekend or a weekday
                if day in ['Saturday', 'Sunday']:  # Apply a weekend discount
                    discount = 0.7  # 30% discount for weekends
                else:  # Apply the weekday discount based on the time range
                    discount = weekday_discount

                # Loop through each vehicle type and apply the corresponding discount
                for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
                    new_row[vehicle] *= discount  # Multiply the vehicle toll by the discount

                # Convert 'id_start' and 'id_end' to integers (in case they are not already)
                new_row['id_start'] = int(new_row['id_start'])
                new_row['id_end'] = int(new_row['id_end'])
                
                # Append the modified row to the list of new rows
                new_rows.append(new_row)

    # Convert the list of new rows into a DataFrame
    result_df = pd.DataFrame(new_rows)

    # Reorganize the columns in the desired order
    result_df = result_df[['id_start', 'start_time', 'start_day', 'id_end', 'end_time', 'end_day', 'distance', 'moto', 'car', 'rv', 'bus', 'truck']]
    
    return result_df  # Return the final DataFrame with time-based toll rates


In [11]:
time_based_toll_df = calculate_time_based_toll_rates(toll_df)
print("\nTime-Based Toll Rates:")
print(time_based_toll_df)


Time-Based Toll Rates:
      id_start start_time start_day   id_end  end_time   end_day  distance  \
0      1001400   00:00:00    Monday  1001402  10:00:00    Monday       9.7   
0      1001400   10:00:00    Monday  1001402  18:00:00    Monday       9.7   
0      1001400   18:00:00    Monday  1001402  23:59:59    Monday       9.7   
0      1001400   00:00:00   Tuesday  1001402  10:00:00   Tuesday       9.7   
0      1001400   10:00:00   Tuesday  1001402  18:00:00   Tuesday       9.7   
...        ...        ...       ...      ...       ...       ...       ...   
1805   1001472   10:00:00  Saturday  1001437  18:00:00  Saturday     202.2   
1805   1001472   18:00:00  Saturday  1001437  23:59:59  Saturday     202.2   
1805   1001472   00:00:00    Sunday  1001437  10:00:00    Sunday     202.2   
1805   1001472   10:00:00    Sunday  1001437  18:00:00    Sunday     202.2   
1805   1001472   18:00:00    Sunday  1001437  23:59:59    Sunday     202.2   

         moto      car      rv      bus